# Feature Engineering en SQL

Creación de nuevas variables sobre datasets preprocesados, en distintos periódos:

- 6 meses, con data drifting menos exigente
- Agrego lag1 y delta1 para las variables que no se procesan como "conceptuales"

In [1]:
# %pip install duckdb
# %pip install jupysql
# %pip install duckdb-engine

In [2]:
# Librerias

import duckdb
import pandas as pd

###########
# funciones

#######
# rutas
from config import dataset_file_prepro_6x, \
                   dataset_file_fe6_6xxpqt

from config import variables_x

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

# Create a connection object
connection = duckdb.connect()

# Pass the connection to the SQL magic command
%sql duckdb:///:memory:

ModuleNotFoundError: No module named 'idna'

In [ ]:
%%sql
create or replace table competencia_03 as
select
    *
from read_parquet("{{dataset_file_prepro_6x}}")

Definir la macro para sumar sin valores nulos


In [ ]:
%%sql
CREATE OR REPLACE MACRO suma_sin_null(a, b) AS ifnull(a, 0) + ifnull(b, 0);

Crear nueva tabla con todos los atributos originales más los nuevos de ingeniería de variables

https://chatgpt.com/share/66edc29b-e0a4-8004-aa86-508d1e7eefa5


Además de agregar las restantes variables según

In [3]:
for var in variables_x:
    print(f"LAG({var}, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_{var},")
    print(f"LAG({var}, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_{var},")
    print(f"{var} - LAG({var}, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_{var},")
    print(f"{var} - LAG({var}, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_{var},")
    print()  # Línea en blanco para separar las variables

LAG(active_quarter, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_active_quarter,
LAG(active_quarter, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_active_quarter,
active_quarter - LAG(active_quarter, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_active_quarter,
active_quarter - LAG(active_quarter, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_active_quarter,

LAG(cliente_vip, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_cliente_vip,
LAG(cliente_vip, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_cliente_vip,
cliente_vip - LAG(cliente_vip, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_cliente_vip,
cliente_vip - LAG(cliente_vip, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_cliente_vip,

LAG(internet, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_internet,
LAG(internet, 2) OVER (PARTI

In [ ]:
%%sql
CREATE OR REPLACE TABLE competencia_03_fe6x_6 AS
WITH base_features AS (
    SELECT
        *,
        suma_sin_null(Master_msaldototal, Visa_msaldototal) AS tc_saldo_total,
        suma_sin_null(mcuenta_corriente, mcaja_ahorro) AS saldo_total_cuentas,
        suma_sin_null(
            suma_sin_null(Visa_msaldototal, Master_msaldototal),
            suma_sin_null(mcuenta_corriente, mcaja_ahorro)
        ) AS saldo_total_completo,
        suma_sin_null(mprestamos_prendarios, mprestamos_hipotecarios) AS deuda_total_prestamos,

        LAG(mpayroll, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mpayroll,
        LAG(mpayroll, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mpayroll,
        LAG(mpayroll, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_mpayroll,
        LAG(mpayroll, 4) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_4_mpayroll,
        LAG(mpayroll, 5) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_5_mpayroll,
        LAG(mpayroll, 6) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_6_mpayroll,

        LAG(suma_sin_null(mtarjeta_visa_consumo, mtarjeta_master_consumo), 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_tarjetas_consumo,
        LAG(suma_sin_null(mtarjeta_visa_consumo, mtarjeta_master_consumo), 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_tarjetas_consumo,
        LAG(suma_sin_null(mtarjeta_visa_consumo, mtarjeta_master_consumo), 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_tarjetas_consumo,
        LAG(suma_sin_null(mtarjeta_visa_consumo, mtarjeta_master_consumo), 4) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_4_tarjetas_consumo,
        LAG(suma_sin_null(mtarjeta_visa_consumo, mtarjeta_master_consumo), 5) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_5_tarjetas_consumo,
        LAG(suma_sin_null(mtarjeta_visa_consumo, mtarjeta_master_consumo), 6) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_6_tarjetas_consumo,

        LAG(mcuenta_corriente, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_cuenta_corriente,
        LAG(mcuenta_corriente, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_cuenta_corriente,
        LAG(mcuenta_corriente, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_cuenta_corriente,
        LAG(mcuenta_corriente, 4) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_4_cuenta_corriente,
        LAG(mcuenta_corriente, 5) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_5_cuenta_corriente,
        LAG(mcuenta_corriente, 6) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_6_cuenta_corriente,

        LAG(mcaja_ahorro, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_caja_ahorro,
        LAG(mcaja_ahorro, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_caja_ahorro,
        LAG(mcaja_ahorro, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_caja_ahorro,
        LAG(mcaja_ahorro, 4) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_4_caja_ahorro,
        LAG(mcaja_ahorro, 5) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_5_caja_ahorro,
        LAG(mcaja_ahorro, 6) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_6_caja_ahorro,

        LAG(mcuentas_saldo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_cuentas_saldo,
        LAG(mcuentas_saldo, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_cuentas_saldo,
        LAG(mcuentas_saldo, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_cuentas_saldo,
        LAG(mcuentas_saldo, 4) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_4_cuentas_saldo,
        LAG(mcuentas_saldo, 5) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_5_cuentas_saldo,
        LAG(mcuentas_saldo, 6) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_6_cuentas_saldo,

        LAG(mcomisiones_mantenimiento, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_comisiones_mantenimiento,
        LAG(mcomisiones_mantenimiento, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_comisiones_mantenimiento,
        LAG(mcomisiones_mantenimiento, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_comisiones_mantenimiento,
        LAG(mcomisiones_mantenimiento, 4) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_4_comisiones_mantenimiento,
        LAG(mcomisiones_mantenimiento, 5) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_5_comisiones_mantenimiento,
        LAG(mcomisiones_mantenimiento, 6) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_6_comisiones_mantenimiento,

        LAG(mcomisiones_otras, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_comisiones_otras,
        LAG(mcomisiones_otras, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_comisiones_otras,
        LAG(mcomisiones_otras, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_3_comisiones_otras,
        LAG(mcomisiones_otras, 4) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_4_comisiones_otras,
        LAG(mcomisiones_otras, 5) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_5_comisiones_otras,
        LAG(mcomisiones_otras, 6) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_6_comisiones_otras,

        mpayroll - LAG(mpayroll, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_mpayroll,
        mpayroll - LAG(mpayroll, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_mpayroll,
        mpayroll - LAG(mpayroll, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_3_mpayroll,
        mpayroll - LAG(mpayroll, 4) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_4_mpayroll,
        mpayroll - LAG(mpayroll, 5) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_5_mpayroll,
        mpayroll - LAG(mpayroll, 6) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_6_mpayroll,

        mtarjeta_visa_consumo - LAG(mtarjeta_visa_consumo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_visa_consumo,
        mtarjeta_visa_consumo - LAG(mtarjeta_visa_consumo, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_visa_consumo,
        mtarjeta_visa_consumo - LAG(mtarjeta_visa_consumo, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_3_visa_consumo,
        mtarjeta_visa_consumo - LAG(mtarjeta_visa_consumo, 4) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_4_visa_consumo,
        mtarjeta_visa_consumo - LAG(mtarjeta_visa_consumo, 5) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_5_visa_consumo,
        mtarjeta_visa_consumo - LAG(mtarjeta_visa_consumo, 6) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_6_visa_consumo,

        mcuenta_corriente - LAG(mcuenta_corriente, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_cuenta_corriente,
        mcuenta_corriente - LAG(mcuenta_corriente, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_cuenta_corriente,
        mcuenta_corriente - LAG(mcuenta_corriente, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_3_cuenta_corriente,
        mcuenta_corriente - LAG(mcuenta_corriente, 4) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_4_cuenta_corriente,
        mcuenta_corriente - LAG(mcuenta_corriente, 5) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_5_cuenta_corriente,
        mcuenta_corriente - LAG(mcuenta_corriente, 6) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_6_cuenta_corriente,

        ctarjeta_visa_transacciones - LAG(ctarjeta_visa_transacciones, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_transacciones_visa,
        ctarjeta_visa_transacciones - LAG(ctarjeta_visa_transacciones, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_transacciones_visa,
        ctarjeta_visa_transacciones - LAG(ctarjeta_visa_transacciones, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_3_transacciones_visa,
        ctarjeta_visa_transacciones - LAG(ctarjeta_visa_transacciones, 4) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_4_transacciones_visa,
        ctarjeta_visa_transacciones - LAG(ctarjeta_visa_transacciones, 5) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_5_transacciones_visa,
        ctarjeta_visa_transacciones - LAG(ctarjeta_visa_transacciones, 6) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_6_transacciones_visa,

        mcomisiones_mantenimiento - LAG(mcomisiones_mantenimiento, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_comisiones_mantenimiento,
        mcomisiones_mantenimiento - LAG(mcomisiones_mantenimiento, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_comisiones_mantenimiento,
        mcomisiones_mantenimiento - LAG(mcomisiones_mantenimiento, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_3_comisiones_mantenimiento,
        mcomisiones_mantenimiento - LAG(mcomisiones_mantenimiento, 4) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_4_comisiones_mantenimiento,
        mcomisiones_mantenimiento - LAG(mcomisiones_mantenimiento, 5) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_5_comisiones_mantenimiento,
        mcomisiones_mantenimiento - LAG(mcomisiones_mantenimiento, 6) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_6_comisiones_mantenimiento,

        mcomisiones_otras - LAG(mcomisiones_otras, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_comisiones_otras,
        mcomisiones_otras - LAG(mcomisiones_otras, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_comisiones_otras,
        mcomisiones_otras - LAG(mcomisiones_otras, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_3_comisiones_otras,
        mcomisiones_otras - LAG(mcomisiones_otras, 4) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_4_comisiones_otras,
        mcomisiones_otras - LAG(mcomisiones_otras, 5) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_5_comisiones_otras,
        mcomisiones_otras - LAG(mcomisiones_otras, 6) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_6_comisiones_otras,

       REGR_SLOPE(mpayroll, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 1 PRECEDING AND 1 PRECEDING
        ) AS payroll_slope_1_foto_mes,

        REGR_SLOPE(mpayroll, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING
        ) AS payroll_slope_2_foto_mes,

        REGR_SLOPE(mpayroll, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 3 PRECEDING AND 1 PRECEDING
        ) AS payroll_slope_3_foto_mes,

        REGR_SLOPE(mpayroll, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 4 PRECEDING AND 1 PRECEDING
        ) AS payroll_slope_4_foto_mes,

        REGR_SLOPE(mpayroll, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING
        ) AS payroll_slope_5_foto_mes,

        REGR_SLOPE(mpayroll, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 6 PRECEDING AND 1 PRECEDING
        ) AS payroll_slope_6_foto_mes,

        REGR_SLOPE(mcuenta_corriente, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 1 PRECEDING AND 1 PRECEDING
        ) AS cuenta_corriente_slope_1_foto_mes,

        REGR_SLOPE(mcuenta_corriente, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING
        ) AS cuenta_corriente_slope_2_foto_mes,

        REGR_SLOPE(mcuenta_corriente, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 3 PRECEDING AND 1 PRECEDING
        ) AS cuenta_corriente_slope_3_foto_mes,

        REGR_SLOPE(mcuenta_corriente, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 4 PRECEDING AND 1 PRECEDING
        ) AS cuenta_corriente_slope_4_foto_mes,

        REGR_SLOPE(mcuenta_corriente, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING
        ) AS cuenta_corriente_slope_5_foto_mes,

        REGR_SLOPE(mcuenta_corriente, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 6 PRECEDING AND 1 PRECEDING
        ) AS cuenta_corriente_slope_6_foto_mes,

        REGR_SLOPE(mtarjeta_visa_consumo, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 1 PRECEDING AND 1 PRECEDING
        ) AS visa_consumo_slope_1_foto_mes,

        REGR_SLOPE(mtarjeta_visa_consumo, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING
        ) AS visa_consumo_slope_2_foto_mes,

        REGR_SLOPE(mtarjeta_visa_consumo, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 3 PRECEDING AND 1 PRECEDING
        ) AS visa_consumo_slope_3_foto_mes,

        REGR_SLOPE(mtarjeta_visa_consumo, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 4 PRECEDING AND 1 PRECEDING
        ) AS visa_consumo_slope_4_foto_mes,

        REGR_SLOPE(mtarjeta_visa_consumo, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING
        ) AS visa_consumo_slope_5_foto_mes,

        REGR_SLOPE(mtarjeta_visa_consumo, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 6 PRECEDING AND 1 PRECEDING
        ) AS visa_consumo_slope_6_foto_mes,

        REGR_SLOPE(mcomisiones_mantenimiento, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 1 PRECEDING AND 1 PRECEDING
        ) AS comisiones_mantenimiento_slope_1_foto_mes,

        REGR_SLOPE(mcomisiones_mantenimiento, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING
        ) AS comisiones_mantenimiento_slope_2_foto_mes,

        REGR_SLOPE(mcomisiones_mantenimiento, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 3 PRECEDING AND 1 PRECEDING
        ) AS comisiones_mantenimiento_slope_3_foto_mes,

        REGR_SLOPE(mcomisiones_mantenimiento, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 4 PRECEDING AND 1 PRECEDING
        ) AS comisiones_mantenimiento_slope_4_foto_mes,

        REGR_SLOPE(mcomisiones_mantenimiento, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING
        ) AS comisiones_mantenimiento_slope_5_foto_mes,

        REGR_SLOPE(mcomisiones_mantenimiento, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 6 PRECEDING AND 1 PRECEDING
        ) AS comisiones_mantenimiento_slope_6_foto_mes,

        REGR_SLOPE(mcomisiones_otras, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 1 PRECEDING AND 1 PRECEDING
        ) AS comisiones_otras_slope_1_foto_mes,

        REGR_SLOPE(mcomisiones_otras, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 2 PRECEDING AND 1 PRECEDING
        ) AS comisiones_otras_slope_2_foto_mes,

        REGR_SLOPE(mcomisiones_otras, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 3 PRECEDING AND 1 PRECEDING
        ) AS comisiones_otras_slope_3_foto_mes,

        REGR_SLOPE(mcomisiones_otras, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 4 PRECEDING AND 1 PRECEDING
        ) AS comisiones_otras_slope_4_foto_mes,

        REGR_SLOPE(mcomisiones_otras, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING
        ) AS comisiones_otras_slope_5_foto_mes,

        REGR_SLOPE(mcomisiones_otras, foto_mes) OVER (
            PARTITION BY numero_de_cliente ORDER BY foto_mes
            ROWS BETWEEN 6 PRECEDING AND 1 PRECEDING
        ) AS comisiones_otras_slope_6_foto_mes,

        CASE WHEN Master_status IN (6, 7, 9) THEN 1 ELSE 0 END AS Master_cuenta_cerrada,
        CASE WHEN Visa_status IN (6, 7, 9) THEN 1 ELSE 0 END AS Visa_cuenta_cerrada,

        mcuentas_saldo / NULLIF(suma_sin_null(mactivos_margen, mpasivos_margen), 0) AS ratio_saldo_margen,
        suma_sin_null(mcomisiones_mantenimiento, mcomisiones_otras) AS total_comisiones_cliente,

        suma_sin_null(ctarjeta_visa_transacciones, ctarjeta_master_transacciones) AS transacciones_totales_tarjetas,
        ctarjeta_debito_transacciones - suma_sin_null(ctarjeta_visa_transacciones, ctarjeta_master_transacciones) AS delta_debito_vs_credito,
        thomebanking + tmobile_app AS indicador_banca_digital,

        LAG(active_quarter, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_active_quarter,
        LAG(active_quarter, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_active_quarter,
        active_quarter - LAG(active_quarter, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_active_quarter,
        active_quarter - LAG(active_quarter, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_active_quarter,

        LAG(cliente_vip, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_cliente_vip,
        LAG(cliente_vip, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_cliente_vip,
        cliente_vip - LAG(cliente_vip, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_cliente_vip,
        cliente_vip - LAG(cliente_vip, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_cliente_vip,

        LAG(internet, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_internet,
        LAG(internet, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_internet,
        internet - LAG(internet, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_internet,
        internet - LAG(internet, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_internet,

        LAG(cliente_edad, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_cliente_edad,
        LAG(cliente_edad, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_cliente_edad,
        cliente_edad - LAG(cliente_edad, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_cliente_edad,
        cliente_edad - LAG(cliente_edad, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_cliente_edad,

        LAG(cliente_antiguedad, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_cliente_antiguedad,
        LAG(cliente_antiguedad, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_cliente_antiguedad,
        cliente_antiguedad - LAG(cliente_antiguedad, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_cliente_antiguedad,
        cliente_antiguedad - LAG(cliente_antiguedad, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_cliente_antiguedad,

        LAG(mrentabilidad, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mrentabilidad,
        LAG(mrentabilidad, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mrentabilidad,
        mrentabilidad - LAG(mrentabilidad, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_mrentabilidad,
        mrentabilidad - LAG(mrentabilidad, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_mrentabilidad,

        LAG(mrentabilidad_annual, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mrentabilidad_annual,
        LAG(mrentabilidad_annual, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mrentabilidad_annual,
        mrentabilidad_annual - LAG(mrentabilidad_annual, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_mrentabilidad_annual,
        mrentabilidad_annual - LAG(mrentabilidad_annual, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_mrentabilidad_annual,

        LAG(mcomisiones, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mcomisiones,
        LAG(mcomisiones, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mcomisiones,
        mcomisiones - LAG(mcomisiones, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_mcomisiones,
        mcomisiones - LAG(mcomisiones, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_mcomisiones,

        LAG(mactivos_margen, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mactivos_margen,
        LAG(mactivos_margen, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mactivos_margen,
        mactivos_margen - LAG(mactivos_margen, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_mactivos_margen,
        mactivos_margen - LAG(mactivos_margen, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_mactivos_margen,

        LAG(mpasivos_margen, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mpasivos_margen,
        LAG(mpasivos_margen, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mpasivos_margen,
        mpasivos_margen - LAG(mpasivos_margen, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_mpasivos_margen,
        mpasivos_margen - LAG(mpasivos_margen, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_mpasivos_margen,

        LAG(cproductos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_cproductos,
        LAG(cproductos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_cproductos,
        cproductos - LAG(cproductos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_cproductos,
        cproductos - LAG(cproductos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_cproductos,

        LAG(tcuentas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_tcuentas,
        LAG(tcuentas, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_tcuentas,
        tcuentas - LAG(tcuentas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_tcuentas,
        tcuentas - LAG(tcuentas, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_tcuentas,

        LAG(ccuenta_corriente, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_ccuenta_corriente,
        LAG(ccuenta_corriente, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_ccuenta_corriente,
        ccuenta_corriente - LAG(ccuenta_corriente, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_ccuenta_corriente,
        ccuenta_corriente - LAG(ccuenta_corriente, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_ccuenta_corriente,

        LAG(mcuenta_corriente_adicional, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mcuenta_corriente_adicional,
        LAG(mcuenta_corriente_adicional, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mcuenta_corriente_adicional,
        mcuenta_corriente_adicional - LAG(mcuenta_corriente_adicional, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_mcuenta_corriente_adicional,
        mcuenta_corriente_adicional - LAG(mcuenta_corriente_adicional, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_mcuenta_corriente_adicional,

        LAG(ccaja_ahorro, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_ccaja_ahorro,
        LAG(ccaja_ahorro, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_ccaja_ahorro,
        ccaja_ahorro - LAG(ccaja_ahorro, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_ccaja_ahorro,
        ccaja_ahorro - LAG(ccaja_ahorro, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_ccaja_ahorro,

        LAG(mcaja_ahorro_adicional, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mcaja_ahorro_adicional,
        LAG(mcaja_ahorro_adicional, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mcaja_ahorro_adicional,
        mcaja_ahorro_adicional - LAG(mcaja_ahorro_adicional, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_mcaja_ahorro_adicional,
        mcaja_ahorro_adicional - LAG(mcaja_ahorro_adicional, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_mcaja_ahorro_adicional,

        LAG(mcaja_ahorro_dolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mcaja_ahorro_dolares,
        LAG(mcaja_ahorro_dolares, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mcaja_ahorro_dolares,
        mcaja_ahorro_dolares - LAG(mcaja_ahorro_dolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_mcaja_ahorro_dolares,
        mcaja_ahorro_dolares - LAG(mcaja_ahorro_dolares, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_mcaja_ahorro_dolares,

        LAG(cdescubierto_preacordado, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_cdescubierto_preacordado,
        LAG(cdescubierto_preacordado, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_cdescubierto_preacordado,
        cdescubierto_preacordado - LAG(cdescubierto_preacordado, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_cdescubierto_preacordado,
        cdescubierto_preacordado - LAG(cdescubierto_preacordado, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_cdescubierto_preacordado,

        LAG(ctarjeta_debito, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_ctarjeta_debito,
        LAG(ctarjeta_debito, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_ctarjeta_debito,
        ctarjeta_debito - LAG(ctarjeta_debito, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_ctarjeta_debito,
        ctarjeta_debito - LAG(ctarjeta_debito, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_ctarjeta_debito,

        LAG(ctarjeta_debito_transacciones, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_ctarjeta_debito_transacciones,
        LAG(ctarjeta_debito_transacciones, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_ctarjeta_debito_transacciones,
        ctarjeta_debito_transacciones - LAG(ctarjeta_debito_transacciones, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_ctarjeta_debito_transacciones,
        ctarjeta_debito_transacciones - LAG(ctarjeta_debito_transacciones, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_ctarjeta_debito_transacciones,

        LAG(mautoservicio, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mautoservicio,
        LAG(mautoservicio, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mautoservicio,
        mautoservicio - LAG(mautoservicio, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_mautoservicio,
        mautoservicio - LAG(mautoservicio, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_mautoservicio,

        LAG(ctarjeta_visa, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_ctarjeta_visa,
        LAG(ctarjeta_visa, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_ctarjeta_visa,
        ctarjeta_visa - LAG(ctarjeta_visa, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_ctarjeta_visa,
        ctarjeta_visa - LAG(ctarjeta_visa, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_ctarjeta_visa,

        LAG(ctarjeta_visa_transacciones, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_ctarjeta_visa_transacciones,
        LAG(ctarjeta_visa_transacciones, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_ctarjeta_visa_transacciones,
        ctarjeta_visa_transacciones - LAG(ctarjeta_visa_transacciones, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_ctarjeta_visa_transacciones,
        ctarjeta_visa_transacciones - LAG(ctarjeta_visa_transacciones, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_ctarjeta_visa_transacciones,

        LAG(ctarjeta_master, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_ctarjeta_master,
        LAG(ctarjeta_master, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_ctarjeta_master,
        ctarjeta_master - LAG(ctarjeta_master, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_ctarjeta_master,
        ctarjeta_master - LAG(ctarjeta_master, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_ctarjeta_master,

        LAG(ctarjeta_master_transacciones, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_ctarjeta_master_transacciones,
        LAG(ctarjeta_master_transacciones, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_ctarjeta_master_transacciones,
        ctarjeta_master_transacciones - LAG(ctarjeta_master_transacciones, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_ctarjeta_master_transacciones,
        ctarjeta_master_transacciones - LAG(ctarjeta_master_transacciones, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_ctarjeta_master_transacciones,

        LAG(cprestamos_prendarios, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_cprestamos_prendarios,
        LAG(cprestamos_prendarios, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_cprestamos_prendarios,
        cprestamos_prendarios - LAG(cprestamos_prendarios, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_cprestamos_prendarios,
        cprestamos_prendarios - LAG(cprestamos_prendarios, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_cprestamos_prendarios,

        LAG(mprestamos_prendarios, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mprestamos_prendarios,
        LAG(mprestamos_prendarios, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mprestamos_prendarios,
        mprestamos_prendarios - LAG(mprestamos_prendarios, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_mprestamos_prendarios,
        mprestamos_prendarios - LAG(mprestamos_prendarios, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_mprestamos_prendarios,

        LAG(cprestamos_hipotecarios, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_cprestamos_hipotecarios,
        LAG(cprestamos_hipotecarios, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_cprestamos_hipotecarios,
        cprestamos_hipotecarios - LAG(cprestamos_hipotecarios, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_cprestamos_hipotecarios,
        cprestamos_hipotecarios - LAG(cprestamos_hipotecarios, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_cprestamos_hipotecarios,

        LAG(mprestamos_hipotecarios, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mprestamos_hipotecarios,
        LAG(mprestamos_hipotecarios, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mprestamos_hipotecarios,
        mprestamos_hipotecarios - LAG(mprestamos_hipotecarios, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_mprestamos_hipotecarios,
        mprestamos_hipotecarios - LAG(mprestamos_hipotecarios, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_mprestamos_hipotecarios,

        LAG(cplazo_fijo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_cplazo_fijo,
        LAG(cplazo_fijo, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_cplazo_fijo,
        cplazo_fijo - LAG(cplazo_fijo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_cplazo_fijo,
        cplazo_fijo - LAG(cplazo_fijo, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_cplazo_fijo,

        LAG(mplazo_fijo_dolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mplazo_fijo_dolares,
        LAG(mplazo_fijo_dolares, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mplazo_fijo_dolares,
        mplazo_fijo_dolares - LAG(mplazo_fijo_dolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_mplazo_fijo_dolares,
        mplazo_fijo_dolares - LAG(mplazo_fijo_dolares, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_mplazo_fijo_dolares,

        LAG(mplazo_fijo_pesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mplazo_fijo_pesos,
        LAG(mplazo_fijo_pesos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mplazo_fijo_pesos,
        mplazo_fijo_pesos - LAG(mplazo_fijo_pesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_mplazo_fijo_pesos,
        mplazo_fijo_pesos - LAG(mplazo_fijo_pesos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_mplazo_fijo_pesos,

        LAG(cinversion1, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_cinversion1,
        LAG(cinversion1, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_cinversion1,
        cinversion1 - LAG(cinversion1, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_cinversion1,
        cinversion1 - LAG(cinversion1, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_cinversion1,

        LAG(minversion1_pesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_minversion1_pesos,
        LAG(minversion1_pesos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_minversion1_pesos,
        minversion1_pesos - LAG(minversion1_pesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_minversion1_pesos,
        minversion1_pesos - LAG(minversion1_pesos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_minversion1_pesos,

        LAG(minversion1_dolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_minversion1_dolares,
        LAG(minversion1_dolares, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_minversion1_dolares,
        minversion1_dolares - LAG(minversion1_dolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_minversion1_dolares,
        minversion1_dolares - LAG(minversion1_dolares, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_minversion1_dolares,

        LAG(cinversion2, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_cinversion2,
        LAG(cinversion2, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_cinversion2,
        cinversion2 - LAG(cinversion2, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_cinversion2,
        cinversion2 - LAG(cinversion2, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_cinversion2,

        LAG(minversion2, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_minversion2,
        LAG(minversion2, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_minversion2,
        minversion2 - LAG(minversion2, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_minversion2,
        minversion2 - LAG(minversion2, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_minversion2,

        LAG(cseguro_vida, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_cseguro_vida,
        LAG(cseguro_vida, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_cseguro_vida,
        cseguro_vida - LAG(cseguro_vida, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_cseguro_vida,
        cseguro_vida - LAG(cseguro_vida, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_cseguro_vida,

        LAG(cseguro_auto, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_cseguro_auto,
        LAG(cseguro_auto, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_cseguro_auto,
        cseguro_auto - LAG(cseguro_auto, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_cseguro_auto,
        cseguro_auto - LAG(cseguro_auto, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_cseguro_auto,

        LAG(cseguro_vivienda, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_cseguro_vivienda,
        LAG(cseguro_vivienda, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_cseguro_vivienda,
        cseguro_vivienda - LAG(cseguro_vivienda, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_cseguro_vivienda,
        cseguro_vivienda - LAG(cseguro_vivienda, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_cseguro_vivienda,

        LAG(cseguro_accidentes_personales, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_cseguro_accidentes_personales,
        LAG(cseguro_accidentes_personales, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_cseguro_accidentes_personales,
        cseguro_accidentes_personales - LAG(cseguro_accidentes_personales, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_cseguro_accidentes_personales,
        cseguro_accidentes_personales - LAG(cseguro_accidentes_personales, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_cseguro_accidentes_personales,

        LAG(ccaja_seguridad, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_ccaja_seguridad,
        LAG(ccaja_seguridad, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_ccaja_seguridad,
        ccaja_seguridad - LAG(ccaja_seguridad, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_ccaja_seguridad,
        ccaja_seguridad - LAG(ccaja_seguridad, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_ccaja_seguridad,

        LAG(cpayroll_trx, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_cpayroll_trx,
        LAG(cpayroll_trx, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_cpayroll_trx,
        cpayroll_trx - LAG(cpayroll_trx, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_cpayroll_trx,
        cpayroll_trx - LAG(cpayroll_trx, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_cpayroll_trx,

        LAG(mpayroll2, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mpayroll2,
        LAG(mpayroll2, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mpayroll2,
        mpayroll2 - LAG(mpayroll2, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_mpayroll2,
        mpayroll2 - LAG(mpayroll2, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_mpayroll2,

        LAG(cpayroll2_trx, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_cpayroll2_trx,
        LAG(cpayroll2_trx, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_cpayroll2_trx,
        cpayroll2_trx - LAG(cpayroll2_trx, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_cpayroll2_trx,
        cpayroll2_trx - LAG(cpayroll2_trx, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_cpayroll2_trx,

        LAG(ccuenta_debitos_automaticos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_ccuenta_debitos_automaticos,
        LAG(ccuenta_debitos_automaticos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_ccuenta_debitos_automaticos,
        ccuenta_debitos_automaticos - LAG(ccuenta_debitos_automaticos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_ccuenta_debitos_automaticos,
        ccuenta_debitos_automaticos - LAG(ccuenta_debitos_automaticos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_ccuenta_debitos_automaticos,

        LAG(mcuenta_debitos_automaticos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mcuenta_debitos_automaticos,
        LAG(mcuenta_debitos_automaticos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mcuenta_debitos_automaticos,
        mcuenta_debitos_automaticos - LAG(mcuenta_debitos_automaticos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_mcuenta_debitos_automaticos,
        mcuenta_debitos_automaticos - LAG(mcuenta_debitos_automaticos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_mcuenta_debitos_automaticos,

        LAG(ctarjeta_visa_debitos_automaticos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_ctarjeta_visa_debitos_automaticos,
        LAG(ctarjeta_visa_debitos_automaticos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_ctarjeta_visa_debitos_automaticos,
        ctarjeta_visa_debitos_automaticos - LAG(ctarjeta_visa_debitos_automaticos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_ctarjeta_visa_debitos_automaticos,
        ctarjeta_visa_debitos_automaticos - LAG(ctarjeta_visa_debitos_automaticos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_ctarjeta_visa_debitos_automaticos,

        LAG(ctarjeta_master_debitos_automaticos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_ctarjeta_master_debitos_automaticos,
        LAG(ctarjeta_master_debitos_automaticos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_ctarjeta_master_debitos_automaticos,
        ctarjeta_master_debitos_automaticos - LAG(ctarjeta_master_debitos_automaticos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_ctarjeta_master_debitos_automaticos,
        ctarjeta_master_debitos_automaticos - LAG(ctarjeta_master_debitos_automaticos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_ctarjeta_master_debitos_automaticos,

        LAG(mttarjeta_master_debitos_automaticos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mttarjeta_master_debitos_automaticos,
        LAG(mttarjeta_master_debitos_automaticos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mttarjeta_master_debitos_automaticos,
        mttarjeta_master_debitos_automaticos - LAG(mttarjeta_master_debitos_automaticos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_mttarjeta_master_debitos_automaticos,
        mttarjeta_master_debitos_automaticos - LAG(mttarjeta_master_debitos_automaticos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_mttarjeta_master_debitos_automaticos,

        LAG(cpagodeservicios, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_cpagodeservicios,
        LAG(cpagodeservicios, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_cpagodeservicios,
        cpagodeservicios - LAG(cpagodeservicios, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_cpagodeservicios,
        cpagodeservicios - LAG(cpagodeservicios, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_cpagodeservicios,

        LAG(mpagodeservicios, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mpagodeservicios,
        LAG(mpagodeservicios, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mpagodeservicios,
        mpagodeservicios - LAG(mpagodeservicios, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_mpagodeservicios,
        mpagodeservicios - LAG(mpagodeservicios, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_mpagodeservicios,

        LAG(cpagomiscuentas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_cpagomiscuentas,
        LAG(cpagomiscuentas, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_cpagomiscuentas,
        cpagomiscuentas - LAG(cpagomiscuentas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_cpagomiscuentas,
        cpagomiscuentas - LAG(cpagomiscuentas, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_cpagomiscuentas,

        LAG(mpagomiscuentas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mpagomiscuentas,
        LAG(mpagomiscuentas, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mpagomiscuentas,
        mpagomiscuentas - LAG(mpagomiscuentas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_mpagomiscuentas,
        mpagomiscuentas - LAG(mpagomiscuentas, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_mpagomiscuentas,

        LAG(ccajeros_propios_descuentos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_ccajeros_propios_descuentos,
        LAG(ccajeros_propios_descuentos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_ccajeros_propios_descuentos,
        ccajeros_propios_descuentos - LAG(ccajeros_propios_descuentos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_ccajeros_propios_descuentos,
        ccajeros_propios_descuentos - LAG(ccajeros_propios_descuentos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_ccajeros_propios_descuentos,

        LAG(mcajeros_propios_descuentos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mcajeros_propios_descuentos,
        LAG(mcajeros_propios_descuentos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mcajeros_propios_descuentos,
        mcajeros_propios_descuentos - LAG(mcajeros_propios_descuentos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_mcajeros_propios_descuentos,
        mcajeros_propios_descuentos - LAG(mcajeros_propios_descuentos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_mcajeros_propios_descuentos,

        LAG(ctarjeta_visa_descuentos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_ctarjeta_visa_descuentos,
        LAG(ctarjeta_visa_descuentos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_ctarjeta_visa_descuentos,
        ctarjeta_visa_descuentos - LAG(ctarjeta_visa_descuentos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_ctarjeta_visa_descuentos,
        ctarjeta_visa_descuentos - LAG(ctarjeta_visa_descuentos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_ctarjeta_visa_descuentos,

        LAG(mtarjeta_visa_descuentos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mtarjeta_visa_descuentos,
        LAG(mtarjeta_visa_descuentos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mtarjeta_visa_descuentos,
        mtarjeta_visa_descuentos - LAG(mtarjeta_visa_descuentos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_mtarjeta_visa_descuentos,
        mtarjeta_visa_descuentos - LAG(mtarjeta_visa_descuentos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_mtarjeta_visa_descuentos,

        LAG(ctarjeta_master_descuentos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_ctarjeta_master_descuentos,
        LAG(ctarjeta_master_descuentos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_ctarjeta_master_descuentos,
        ctarjeta_master_descuentos - LAG(ctarjeta_master_descuentos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_ctarjeta_master_descuentos,
        ctarjeta_master_descuentos - LAG(ctarjeta_master_descuentos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_ctarjeta_master_descuentos,

        LAG(mtarjeta_master_descuentos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mtarjeta_master_descuentos,
        LAG(mtarjeta_master_descuentos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mtarjeta_master_descuentos,
        mtarjeta_master_descuentos - LAG(mtarjeta_master_descuentos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_mtarjeta_master_descuentos,
        mtarjeta_master_descuentos - LAG(mtarjeta_master_descuentos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_mtarjeta_master_descuentos,

        LAG(ccomisiones_mantenimiento, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_ccomisiones_mantenimiento,
        LAG(ccomisiones_mantenimiento, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_ccomisiones_mantenimiento,
        ccomisiones_mantenimiento - LAG(ccomisiones_mantenimiento, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_ccomisiones_mantenimiento,
        ccomisiones_mantenimiento - LAG(ccomisiones_mantenimiento, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_ccomisiones_mantenimiento,

        LAG(ccomisiones_otras, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_ccomisiones_otras,
        LAG(ccomisiones_otras, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_ccomisiones_otras,
        ccomisiones_otras - LAG(ccomisiones_otras, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_ccomisiones_otras,
        ccomisiones_otras - LAG(ccomisiones_otras, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_ccomisiones_otras,

        LAG(cforex, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_cforex,
        LAG(cforex, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_cforex,
        cforex - LAG(cforex, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_cforex,
        cforex - LAG(cforex, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_cforex,

        LAG(cforex_buy, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_cforex_buy,
        LAG(cforex_buy, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_cforex_buy,
        cforex_buy - LAG(cforex_buy, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_cforex_buy,
        cforex_buy - LAG(cforex_buy, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_cforex_buy,

        LAG(mforex_buy, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mforex_buy,
        LAG(mforex_buy, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mforex_buy,
        mforex_buy - LAG(mforex_buy, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_mforex_buy,
        mforex_buy - LAG(mforex_buy, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_mforex_buy,

        LAG(cforex_sell, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_cforex_sell,
        LAG(cforex_sell, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_cforex_sell,
        cforex_sell - LAG(cforex_sell, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_cforex_sell,
        cforex_sell - LAG(cforex_sell, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_cforex_sell,

        LAG(mforex_sell, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mforex_sell,
        LAG(mforex_sell, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mforex_sell,
        mforex_sell - LAG(mforex_sell, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_mforex_sell,
        mforex_sell - LAG(mforex_sell, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_mforex_sell,

        LAG(ctransferencias_recibidas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_ctransferencias_recibidas,
        LAG(ctransferencias_recibidas, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_ctransferencias_recibidas,
        ctransferencias_recibidas - LAG(ctransferencias_recibidas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_ctransferencias_recibidas,
        ctransferencias_recibidas - LAG(ctransferencias_recibidas, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_ctransferencias_recibidas,

        LAG(mtransferencias_recibidas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mtransferencias_recibidas,
        LAG(mtransferencias_recibidas, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mtransferencias_recibidas,
        mtransferencias_recibidas - LAG(mtransferencias_recibidas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_mtransferencias_recibidas,
        mtransferencias_recibidas - LAG(mtransferencias_recibidas, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_mtransferencias_recibidas,

        LAG(ctransferencias_emitidas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_ctransferencias_emitidas,
        LAG(ctransferencias_emitidas, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_ctransferencias_emitidas,
        ctransferencias_emitidas - LAG(ctransferencias_emitidas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_ctransferencias_emitidas,
        ctransferencias_emitidas - LAG(ctransferencias_emitidas, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_ctransferencias_emitidas,

        LAG(mtransferencias_emitidas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mtransferencias_emitidas,
        LAG(mtransferencias_emitidas, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mtransferencias_emitidas,
        mtransferencias_emitidas - LAG(mtransferencias_emitidas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_mtransferencias_emitidas,
        mtransferencias_emitidas - LAG(mtransferencias_emitidas, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_mtransferencias_emitidas,

        LAG(cextraccion_autoservicio, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_cextraccion_autoservicio,
        LAG(cextraccion_autoservicio, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_cextraccion_autoservicio,
        cextraccion_autoservicio - LAG(cextraccion_autoservicio, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_cextraccion_autoservicio,
        cextraccion_autoservicio - LAG(cextraccion_autoservicio, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_cextraccion_autoservicio,

        LAG(mextraccion_autoservicio, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mextraccion_autoservicio,
        LAG(mextraccion_autoservicio, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mextraccion_autoservicio,
        mextraccion_autoservicio - LAG(mextraccion_autoservicio, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_mextraccion_autoservicio,
        mextraccion_autoservicio - LAG(mextraccion_autoservicio, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_mextraccion_autoservicio,

        LAG(ccheques_depositados, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_ccheques_depositados,
        LAG(ccheques_depositados, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_ccheques_depositados,
        ccheques_depositados - LAG(ccheques_depositados, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_ccheques_depositados,
        ccheques_depositados - LAG(ccheques_depositados, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_ccheques_depositados,

        LAG(mcheques_depositados, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mcheques_depositados,
        LAG(mcheques_depositados, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mcheques_depositados,
        mcheques_depositados - LAG(mcheques_depositados, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_mcheques_depositados,
        mcheques_depositados - LAG(mcheques_depositados, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_mcheques_depositados,

        LAG(ccheques_emitidos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_ccheques_emitidos,
        LAG(ccheques_emitidos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_ccheques_emitidos,
        ccheques_emitidos - LAG(ccheques_emitidos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_ccheques_emitidos,
        ccheques_emitidos - LAG(ccheques_emitidos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_ccheques_emitidos,

        LAG(mcheques_emitidos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mcheques_emitidos,
        LAG(mcheques_emitidos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mcheques_emitidos,
        mcheques_emitidos - LAG(mcheques_emitidos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_mcheques_emitidos,
        mcheques_emitidos - LAG(mcheques_emitidos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_mcheques_emitidos,

        LAG(ccheques_depositados_rechazados, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_ccheques_depositados_rechazados,
        LAG(ccheques_depositados_rechazados, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_ccheques_depositados_rechazados,
        ccheques_depositados_rechazados - LAG(ccheques_depositados_rechazados, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_ccheques_depositados_rechazados,
        ccheques_depositados_rechazados - LAG(ccheques_depositados_rechazados, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_ccheques_depositados_rechazados,

        LAG(mcheques_depositados_rechazados, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mcheques_depositados_rechazados,
        LAG(mcheques_depositados_rechazados, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mcheques_depositados_rechazados,
        mcheques_depositados_rechazados - LAG(mcheques_depositados_rechazados, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_mcheques_depositados_rechazados,
        mcheques_depositados_rechazados - LAG(mcheques_depositados_rechazados, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_mcheques_depositados_rechazados,

        LAG(ccheques_emitidos_rechazados, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_ccheques_emitidos_rechazados,
        LAG(ccheques_emitidos_rechazados, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_ccheques_emitidos_rechazados,
        ccheques_emitidos_rechazados - LAG(ccheques_emitidos_rechazados, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_ccheques_emitidos_rechazados,
        ccheques_emitidos_rechazados - LAG(ccheques_emitidos_rechazados, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_ccheques_emitidos_rechazados,

        LAG(mcheques_emitidos_rechazados, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_mcheques_emitidos_rechazados,
        LAG(mcheques_emitidos_rechazados, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_mcheques_emitidos_rechazados,
        mcheques_emitidos_rechazados - LAG(mcheques_emitidos_rechazados, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_mcheques_emitidos_rechazados,
        mcheques_emitidos_rechazados - LAG(mcheques_emitidos_rechazados, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_mcheques_emitidos_rechazados,

        LAG(tcallcenter, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_tcallcenter,
        LAG(tcallcenter, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_tcallcenter,
        tcallcenter - LAG(tcallcenter, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_tcallcenter,
        tcallcenter - LAG(tcallcenter, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_tcallcenter,

        LAG(ccallcenter_transacciones, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_ccallcenter_transacciones,
        LAG(ccallcenter_transacciones, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_ccallcenter_transacciones,
        ccallcenter_transacciones - LAG(ccallcenter_transacciones, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_ccallcenter_transacciones,
        ccallcenter_transacciones - LAG(ccallcenter_transacciones, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_ccallcenter_transacciones,

        LAG(thomebanking, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_thomebanking,
        LAG(thomebanking, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_thomebanking,
        thomebanking - LAG(thomebanking, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_thomebanking,
        thomebanking - LAG(thomebanking, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_thomebanking,

        LAG(chomebanking_transacciones, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_chomebanking_transacciones,
        LAG(chomebanking_transacciones, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_chomebanking_transacciones,
        chomebanking_transacciones - LAG(chomebanking_transacciones, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_chomebanking_transacciones,
        chomebanking_transacciones - LAG(chomebanking_transacciones, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_chomebanking_transacciones,

        LAG(ccajas_transacciones, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_ccajas_transacciones,
        LAG(ccajas_transacciones, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_ccajas_transacciones,
        ccajas_transacciones - LAG(ccajas_transacciones, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_ccajas_transacciones,
        ccajas_transacciones - LAG(ccajas_transacciones, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_ccajas_transacciones,

        LAG(ccajas_consultas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_ccajas_consultas,
        LAG(ccajas_consultas, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_ccajas_consultas,
        ccajas_consultas - LAG(ccajas_consultas, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_ccajas_consultas,
        ccajas_consultas - LAG(ccajas_consultas, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_ccajas_consultas,

        LAG(ccajas_depositos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_ccajas_depositos,
        LAG(ccajas_depositos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_ccajas_depositos,
        ccajas_depositos - LAG(ccajas_depositos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_ccajas_depositos,
        ccajas_depositos - LAG(ccajas_depositos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_ccajas_depositos,

        LAG(ccajas_extracciones, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_ccajas_extracciones,
        LAG(ccajas_extracciones, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_ccajas_extracciones,
        ccajas_extracciones - LAG(ccajas_extracciones, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_ccajas_extracciones,
        ccajas_extracciones - LAG(ccajas_extracciones, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_ccajas_extracciones,

        LAG(ccajas_otras, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_ccajas_otras,
        LAG(ccajas_otras, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_ccajas_otras,
        ccajas_otras - LAG(ccajas_otras, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_ccajas_otras,
        ccajas_otras - LAG(ccajas_otras, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_ccajas_otras,

        LAG(catm_trx, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_catm_trx,
        LAG(catm_trx, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_catm_trx,
        catm_trx - LAG(catm_trx, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_catm_trx,
        catm_trx - LAG(catm_trx, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_catm_trx,

        LAG(matm, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_matm,
        LAG(matm, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_matm,
        matm - LAG(matm, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_matm,
        matm - LAG(matm, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_matm,

        LAG(catm_trx_other, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_catm_trx_other,
        LAG(catm_trx_other, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_catm_trx_other,
        catm_trx_other - LAG(catm_trx_other, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_catm_trx_other,
        catm_trx_other - LAG(catm_trx_other, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_catm_trx_other,

        LAG(matm_other, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_matm_other,
        LAG(matm_other, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_matm_other,
        matm_other - LAG(matm_other, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_matm_other,
        matm_other - LAG(matm_other, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_matm_other,

        LAG(ctrx_quarter, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_ctrx_quarter,
        LAG(ctrx_quarter, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_ctrx_quarter,
        ctrx_quarter - LAG(ctrx_quarter, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_ctrx_quarter,
        ctrx_quarter - LAG(ctrx_quarter, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_ctrx_quarter,

        LAG(tmobile_app, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_tmobile_app,
        LAG(tmobile_app, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_tmobile_app,
        tmobile_app - LAG(tmobile_app, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_tmobile_app,
        tmobile_app - LAG(tmobile_app, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_tmobile_app,

        LAG(cmobile_app_trx, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_cmobile_app_trx,
        LAG(cmobile_app_trx, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_cmobile_app_trx,
        cmobile_app_trx - LAG(cmobile_app_trx, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_cmobile_app_trx,
        cmobile_app_trx - LAG(cmobile_app_trx, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_cmobile_app_trx,

        LAG(Master_delinquency, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Master_delinquency,
        LAG(Master_delinquency, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Master_delinquency,
        Master_delinquency - LAG(Master_delinquency, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Master_delinquency,
        Master_delinquency - LAG(Master_delinquency, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Master_delinquency,

        LAG(Master_status, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Master_status,
        LAG(Master_status, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Master_status,
        Master_status - LAG(Master_status, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Master_status,
        Master_status - LAG(Master_status, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Master_status,

        LAG(Master_mfinanciacion_limite, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Master_mfinanciacion_limite,
        LAG(Master_mfinanciacion_limite, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Master_mfinanciacion_limite,
        Master_mfinanciacion_limite - LAG(Master_mfinanciacion_limite, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Master_mfinanciacion_limite,
        Master_mfinanciacion_limite - LAG(Master_mfinanciacion_limite, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Master_mfinanciacion_limite,

        LAG(Master_Fvencimiento, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Master_Fvencimiento,
        LAG(Master_Fvencimiento, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Master_Fvencimiento,
        Master_Fvencimiento - LAG(Master_Fvencimiento, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Master_Fvencimiento,
        Master_Fvencimiento - LAG(Master_Fvencimiento, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Master_Fvencimiento,

        LAG(Master_msaldototal, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Master_msaldototal,
        LAG(Master_msaldototal, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Master_msaldototal,
        Master_msaldototal - LAG(Master_msaldototal, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Master_msaldototal,
        Master_msaldototal - LAG(Master_msaldototal, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Master_msaldototal,

        LAG(Master_msaldopesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Master_msaldopesos,
        LAG(Master_msaldopesos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Master_msaldopesos,
        Master_msaldopesos - LAG(Master_msaldopesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Master_msaldopesos,
        Master_msaldopesos - LAG(Master_msaldopesos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Master_msaldopesos,

        LAG(Master_msaldodolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Master_msaldodolares,
        LAG(Master_msaldodolares, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Master_msaldodolares,
        Master_msaldodolares - LAG(Master_msaldodolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Master_msaldodolares,
        Master_msaldodolares - LAG(Master_msaldodolares, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Master_msaldodolares,

        LAG(Master_mconsumospesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Master_mconsumospesos,
        LAG(Master_mconsumospesos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Master_mconsumospesos,
        Master_mconsumospesos - LAG(Master_mconsumospesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Master_mconsumospesos,
        Master_mconsumospesos - LAG(Master_mconsumospesos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Master_mconsumospesos,

        LAG(Master_mconsumosdolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Master_mconsumosdolares,
        LAG(Master_mconsumosdolares, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Master_mconsumosdolares,
        Master_mconsumosdolares - LAG(Master_mconsumosdolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Master_mconsumosdolares,
        Master_mconsumosdolares - LAG(Master_mconsumosdolares, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Master_mconsumosdolares,

        LAG(Master_mlimitecompra, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Master_mlimitecompra,
        LAG(Master_mlimitecompra, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Master_mlimitecompra,
        Master_mlimitecompra - LAG(Master_mlimitecompra, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Master_mlimitecompra,
        Master_mlimitecompra - LAG(Master_mlimitecompra, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Master_mlimitecompra,

        LAG(Master_madelantopesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Master_madelantopesos,
        LAG(Master_madelantopesos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Master_madelantopesos,
        Master_madelantopesos - LAG(Master_madelantopesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Master_madelantopesos,
        Master_madelantopesos - LAG(Master_madelantopesos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Master_madelantopesos,

        LAG(Master_madelantodolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Master_madelantodolares,
        LAG(Master_madelantodolares, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Master_madelantodolares,
        Master_madelantodolares - LAG(Master_madelantodolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Master_madelantodolares,
        Master_madelantodolares - LAG(Master_madelantodolares, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Master_madelantodolares,

        LAG(Master_mpagado, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Master_mpagado,
        LAG(Master_mpagado, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Master_mpagado,
        Master_mpagado - LAG(Master_mpagado, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Master_mpagado,
        Master_mpagado - LAG(Master_mpagado, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Master_mpagado,

        LAG(Master_mpagospesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Master_mpagospesos,
        LAG(Master_mpagospesos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Master_mpagospesos,
        Master_mpagospesos - LAG(Master_mpagospesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Master_mpagospesos,
        Master_mpagospesos - LAG(Master_mpagospesos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Master_mpagospesos,

        LAG(Master_mpagosdolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Master_mpagosdolares,
        LAG(Master_mpagosdolares, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Master_mpagosdolares,
        Master_mpagosdolares - LAG(Master_mpagosdolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Master_mpagosdolares,
        Master_mpagosdolares - LAG(Master_mpagosdolares, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Master_mpagosdolares,

        LAG(Master_fechaalta, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Master_fechaalta,
        LAG(Master_fechaalta, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Master_fechaalta,
        Master_fechaalta - LAG(Master_fechaalta, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Master_fechaalta,
        Master_fechaalta - LAG(Master_fechaalta, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Master_fechaalta,

        LAG(Master_mconsumototal, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Master_mconsumototal,
        LAG(Master_mconsumototal, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Master_mconsumototal,
        Master_mconsumototal - LAG(Master_mconsumototal, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Master_mconsumototal,
        Master_mconsumototal - LAG(Master_mconsumototal, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Master_mconsumototal,

        LAG(Master_cconsumos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Master_cconsumos,
        LAG(Master_cconsumos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Master_cconsumos,
        Master_cconsumos - LAG(Master_cconsumos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Master_cconsumos,
        Master_cconsumos - LAG(Master_cconsumos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Master_cconsumos,

        LAG(Master_cadelantosefectivo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Master_cadelantosefectivo,
        LAG(Master_cadelantosefectivo, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Master_cadelantosefectivo,
        Master_cadelantosefectivo - LAG(Master_cadelantosefectivo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Master_cadelantosefectivo,
        Master_cadelantosefectivo - LAG(Master_cadelantosefectivo, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Master_cadelantosefectivo,

        LAG(Master_mpagominimo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Master_mpagominimo,
        LAG(Master_mpagominimo, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Master_mpagominimo,
        Master_mpagominimo - LAG(Master_mpagominimo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Master_mpagominimo,
        Master_mpagominimo - LAG(Master_mpagominimo, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Master_mpagominimo,

        LAG(Visa_delinquency, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Visa_delinquency,
        LAG(Visa_delinquency, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Visa_delinquency,
        Visa_delinquency - LAG(Visa_delinquency, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Visa_delinquency,
        Visa_delinquency - LAG(Visa_delinquency, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Visa_delinquency,

        LAG(Visa_status, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Visa_status,
        LAG(Visa_status, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Visa_status,
        Visa_status - LAG(Visa_status, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Visa_status,
        Visa_status - LAG(Visa_status, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Visa_status,

        LAG(Visa_mfinanciacion_limite, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Visa_mfinanciacion_limite,
        LAG(Visa_mfinanciacion_limite, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Visa_mfinanciacion_limite,
        Visa_mfinanciacion_limite - LAG(Visa_mfinanciacion_limite, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Visa_mfinanciacion_limite,
        Visa_mfinanciacion_limite - LAG(Visa_mfinanciacion_limite, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Visa_mfinanciacion_limite,

        LAG(Visa_Fvencimiento, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Visa_Fvencimiento,
        LAG(Visa_Fvencimiento, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Visa_Fvencimiento,
        Visa_Fvencimiento - LAG(Visa_Fvencimiento, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Visa_Fvencimiento,
        Visa_Fvencimiento - LAG(Visa_Fvencimiento, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Visa_Fvencimiento,

        LAG(Visa_msaldototal, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Visa_msaldototal,
        LAG(Visa_msaldototal, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Visa_msaldototal,
        Visa_msaldototal - LAG(Visa_msaldototal, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Visa_msaldototal,
        Visa_msaldototal - LAG(Visa_msaldototal, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Visa_msaldototal,

        LAG(Visa_msaldopesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Visa_msaldopesos,
        LAG(Visa_msaldopesos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Visa_msaldopesos,
        Visa_msaldopesos - LAG(Visa_msaldopesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Visa_msaldopesos,
        Visa_msaldopesos - LAG(Visa_msaldopesos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Visa_msaldopesos,

        LAG(Visa_msaldodolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Visa_msaldodolares,
        LAG(Visa_msaldodolares, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Visa_msaldodolares,
        Visa_msaldodolares - LAG(Visa_msaldodolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Visa_msaldodolares,
        Visa_msaldodolares - LAG(Visa_msaldodolares, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Visa_msaldodolares,

        LAG(Visa_mconsumospesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Visa_mconsumospesos,
        LAG(Visa_mconsumospesos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Visa_mconsumospesos,
        Visa_mconsumospesos - LAG(Visa_mconsumospesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Visa_mconsumospesos,
        Visa_mconsumospesos - LAG(Visa_mconsumospesos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Visa_mconsumospesos,

        LAG(Visa_mconsumosdolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Visa_mconsumosdolares,
        LAG(Visa_mconsumosdolares, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Visa_mconsumosdolares,
        Visa_mconsumosdolares - LAG(Visa_mconsumosdolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Visa_mconsumosdolares,
        Visa_mconsumosdolares - LAG(Visa_mconsumosdolares, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Visa_mconsumosdolares,

        LAG(Visa_mlimitecompra, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Visa_mlimitecompra,
        LAG(Visa_mlimitecompra, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Visa_mlimitecompra,
        Visa_mlimitecompra - LAG(Visa_mlimitecompra, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Visa_mlimitecompra,
        Visa_mlimitecompra - LAG(Visa_mlimitecompra, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Visa_mlimitecompra,

        LAG(Visa_madelantopesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Visa_madelantopesos,
        LAG(Visa_madelantopesos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Visa_madelantopesos,
        Visa_madelantopesos - LAG(Visa_madelantopesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Visa_madelantopesos,
        Visa_madelantopesos - LAG(Visa_madelantopesos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Visa_madelantopesos,

        LAG(Visa_madelantodolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Visa_madelantodolares,
        LAG(Visa_madelantodolares, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Visa_madelantodolares,
        Visa_madelantodolares - LAG(Visa_madelantodolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Visa_madelantodolares,
        Visa_madelantodolares - LAG(Visa_madelantodolares, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Visa_madelantodolares,

        LAG(Visa_mpagado, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Visa_mpagado,
        LAG(Visa_mpagado, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Visa_mpagado,
        Visa_mpagado - LAG(Visa_mpagado, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Visa_mpagado,
        Visa_mpagado - LAG(Visa_mpagado, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Visa_mpagado,

        LAG(Visa_mpagospesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Visa_mpagospesos,
        LAG(Visa_mpagospesos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Visa_mpagospesos,
        Visa_mpagospesos - LAG(Visa_mpagospesos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Visa_mpagospesos,
        Visa_mpagospesos - LAG(Visa_mpagospesos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Visa_mpagospesos,

        LAG(Visa_mpagosdolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Visa_mpagosdolares,
        LAG(Visa_mpagosdolares, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Visa_mpagosdolares,
        Visa_mpagosdolares - LAG(Visa_mpagosdolares, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Visa_mpagosdolares,
        Visa_mpagosdolares - LAG(Visa_mpagosdolares, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Visa_mpagosdolares,

        LAG(Visa_fechaalta, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Visa_fechaalta,
        LAG(Visa_fechaalta, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Visa_fechaalta,
        Visa_fechaalta - LAG(Visa_fechaalta, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Visa_fechaalta,
        Visa_fechaalta - LAG(Visa_fechaalta, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Visa_fechaalta,

        LAG(Visa_mconsumototal, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Visa_mconsumototal,
        LAG(Visa_mconsumototal, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Visa_mconsumototal,
        Visa_mconsumototal - LAG(Visa_mconsumototal, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Visa_mconsumototal,
        Visa_mconsumototal - LAG(Visa_mconsumototal, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Visa_mconsumototal,

        LAG(Visa_cconsumos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Visa_cconsumos,
        LAG(Visa_cconsumos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Visa_cconsumos,
        Visa_cconsumos - LAG(Visa_cconsumos, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Visa_cconsumos,
        Visa_cconsumos - LAG(Visa_cconsumos, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Visa_cconsumos,

        LAG(Visa_cadelantosefectivo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Visa_cadelantosefectivo,
        LAG(Visa_cadelantosefectivo, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Visa_cadelantosefectivo,
        Visa_cadelantosefectivo - LAG(Visa_cadelantosefectivo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Visa_cadelantosefectivo,
        Visa_cadelantosefectivo - LAG(Visa_cadelantosefectivo, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Visa_cadelantosefectivo,

        LAG(Visa_mpagominimo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_1_Visa_mpagominimo,
        LAG(Visa_mpagominimo, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS lag_2_Visa_mpagominimo,
        Visa_mpagominimo - LAG(Visa_mpagominimo, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_1_Visa_mpagominimo,
        Visa_mpagominimo - LAG(Visa_mpagominimo, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS delta_2_Visa_mpagominimo,

    FROM competencia_03
)

SELECT
    *,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY cliente_antiguedad) AS cliente_antiguedad_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY mpayroll) AS payroll_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY mtarjeta_visa_consumo) AS visa_consumo_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY ctarjeta_visa) AS visa_tarjetas_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY mtarjeta_master_consumo) AS master_consumo_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY ctarjeta_master) AS master_tarjetas_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY cproductos) AS productos_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY mcuenta_corriente) AS cuenta_corriente_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY mcaja_ahorro) AS caja_ahorro_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY suma_sin_null(ctarjeta_visa, ctarjeta_master)) AS tarjetas_credito_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY suma_sin_null(mtarjeta_visa_consumo, mtarjeta_master_consumo)) AS tarjetas_consumo_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY mprestamos_hipotecarios) AS prestamos_hipotecarios_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY tc_saldo_total) AS tc_saldo_total_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY saldo_total_cuentas) AS saldo_total_cuentas_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY saldo_total_completo) AS saldo_total_completo_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY deuda_total_prestamos) AS deuda_total_prestamos_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_1_mpayroll) AS lag_1_mpayroll_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_2_mpayroll) AS lag_2_mpayroll_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_3_mpayroll) AS lag_3_mpayroll_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_4_mpayroll) AS lag_4_mpayroll_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_5_mpayroll) AS lag_5_mpayroll_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_6_mpayroll) AS lag_6_mpayroll_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_1_tarjetas_consumo) AS lag_1_tarjetas_consumo_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_2_tarjetas_consumo) AS lag_2_tarjetas_consumo_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_3_tarjetas_consumo) AS lag_3_tarjetas_consumo_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_4_tarjetas_consumo) AS lag_4_tarjetas_consumo_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_5_tarjetas_consumo) AS lag_5_tarjetas_consumo_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_6_tarjetas_consumo) AS lag_6_tarjetas_consumo_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_1_cuenta_corriente) AS lag_1_cuenta_corriente_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_2_cuenta_corriente) AS lag_2_cuenta_corriente_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_3_cuenta_corriente) AS lag_3_cuenta_corriente_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_4_cuenta_corriente) AS lag_4_cuenta_corriente_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_5_cuenta_corriente) AS lag_5_cuenta_corriente_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_6_cuenta_corriente) AS lag_6_cuenta_corriente_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_1_caja_ahorro) AS lag_1_caja_ahorro_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_2_caja_ahorro) AS lag_2_caja_ahorro_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_3_caja_ahorro) AS lag_3_caja_ahorro_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_4_caja_ahorro) AS lag_4_caja_ahorro_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_5_caja_ahorro) AS lag_5_caja_ahorro_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_6_caja_ahorro) AS lag_6_caja_ahorro_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_1_cuentas_saldo) AS lag_1_cuentas_saldo_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_2_cuentas_saldo) AS lag_2_cuentas_saldo_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_3_cuentas_saldo) AS lag_3_cuentas_saldo_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_4_cuentas_saldo) AS lag_4_cuentas_saldo_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_5_cuentas_saldo) AS lag_5_cuentas_saldo_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_6_cuentas_saldo) AS lag_6_cuentas_saldo_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_1_comisiones_mantenimiento) AS lag_1_comisiones_mantenimiento_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_2_comisiones_mantenimiento) AS lag_2_comisiones_mantenimiento_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_3_comisiones_mantenimiento) AS lag_3_comisiones_mantenimiento_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_4_comisiones_mantenimiento) AS lag_4_comisiones_mantenimiento_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_5_comisiones_mantenimiento) AS lag_5_comisiones_mantenimiento_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_6_comisiones_mantenimiento) AS lag_6_comisiones_mantenimiento_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_1_comisiones_otras) AS lag_1_comisiones_otras_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_2_comisiones_otras) AS lag_2_comisiones_otras_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_3_comisiones_otras) AS lag_3_comisiones_otras_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_4_comisiones_otras) AS lag_4_comisiones_otras_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_5_comisiones_otras) AS lag_5_comisiones_otras_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY lag_6_comisiones_otras) AS lag_6_comisiones_otras_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_1_mpayroll) AS delta_1_mpayroll_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_2_mpayroll) AS delta_2_mpayroll_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_3_mpayroll) AS delta_3_mpayroll_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_4_mpayroll) AS delta_4_mpayroll_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_5_mpayroll) AS delta_5_mpayroll_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_6_mpayroll) AS delta_6_mpayroll_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_1_visa_consumo) AS delta_1_visa_consumo_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_2_visa_consumo) AS delta_2_visa_consumo_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_3_visa_consumo) AS delta_3_visa_consumo_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_4_visa_consumo) AS delta_4_visa_consumo_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_5_visa_consumo) AS delta_5_visa_consumo_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_6_visa_consumo) AS delta_6_visa_consumo_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_1_cuenta_corriente) AS delta_1_cuenta_corriente_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_2_cuenta_corriente) AS delta_2_cuenta_corriente_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_3_cuenta_corriente) AS delta_3_cuenta_corriente_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_4_cuenta_corriente) AS delta_4_cuenta_corriente_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_5_cuenta_corriente) AS delta_5_cuenta_corriente_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_6_cuenta_corriente) AS delta_6_cuenta_corriente_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_1_transacciones_visa) AS delta_1_transacciones_visa_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_2_transacciones_visa) AS delta_2_transacciones_visa_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_3_transacciones_visa) AS delta_3_transacciones_visa_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_4_transacciones_visa) AS delta_4_transacciones_visa_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_5_transacciones_visa) AS delta_5_transacciones_visa_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_6_transacciones_visa) AS delta_6_transacciones_visa_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_1_comisiones_mantenimiento) AS delta_1_comisiones_mantenimiento_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_2_comisiones_mantenimiento) AS delta_2_comisiones_mantenimiento_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_3_comisiones_mantenimiento) AS delta_3_comisiones_mantenimiento_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_4_comisiones_mantenimiento) AS delta_4_comisiones_mantenimiento_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_5_comisiones_mantenimiento) AS delta_5_comisiones_mantenimiento_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_6_comisiones_mantenimiento) AS delta_6_comisiones_mantenimiento_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_1_comisiones_otras) AS delta_1_comisiones_otras_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_2_comisiones_otras) AS delta_2_comisiones_otras_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_3_comisiones_otras) AS delta_3_comisiones_otras_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_4_comisiones_otras) AS delta_4_comisiones_otras_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_5_comisiones_otras) AS delta_5_comisiones_otras_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_6_comisiones_otras) AS delta_6_comisiones_otras_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY payroll_slope_1_foto_mes) AS payroll_slope_1_foto_mes_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY payroll_slope_2_foto_mes) AS payroll_slope_2_foto_mes_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY payroll_slope_3_foto_mes) AS payroll_slope_3_foto_mes_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY payroll_slope_4_foto_mes) AS payroll_slope_4_foto_mes_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY payroll_slope_5_foto_mes) AS payroll_slope_5_foto_mes_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY payroll_slope_6_foto_mes) AS payroll_slope_6_foto_mes_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY cuenta_corriente_slope_1_foto_mes) AS cuenta_corriente_slope_1_foto_mes_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY cuenta_corriente_slope_2_foto_mes) AS cuenta_corriente_slope_2_foto_mes_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY cuenta_corriente_slope_3_foto_mes) AS cuenta_corriente_slope_3_foto_mes_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY cuenta_corriente_slope_4_foto_mes) AS cuenta_corriente_slope_4_foto_mes_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY cuenta_corriente_slope_5_foto_mes) AS cuenta_corriente_slope_5_foto_mes_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY cuenta_corriente_slope_6_foto_mes) AS cuenta_corriente_slope_6_foto_mes_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY visa_consumo_slope_1_foto_mes) AS visa_consumo_slope_1_foto_mes_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY visa_consumo_slope_2_foto_mes) AS visa_consumo_slope_2_foto_mes_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY visa_consumo_slope_3_foto_mes) AS visa_consumo_slope_3_foto_mes_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY visa_consumo_slope_4_foto_mes) AS visa_consumo_slope_4_foto_mes_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY visa_consumo_slope_5_foto_mes) AS visa_consumo_slope_5_foto_mes_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY visa_consumo_slope_6_foto_mes) AS visa_consumo_slope_6_foto_mes_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY comisiones_mantenimiento_slope_1_foto_mes) AS comisiones_mantenimiento_slope_1_foto_mes_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY comisiones_mantenimiento_slope_2_foto_mes) AS comisiones_mantenimiento_slope_2_foto_mes_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY comisiones_mantenimiento_slope_3_foto_mes) AS comisiones_mantenimiento_slope_3_foto_mes_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY comisiones_mantenimiento_slope_4_foto_mes) AS comisiones_mantenimiento_slope_4_foto_mes_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY comisiones_mantenimiento_slope_5_foto_mes) AS comisiones_mantenimiento_slope_5_foto_mes_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY comisiones_mantenimiento_slope_6_foto_mes) AS comisiones_mantenimiento_slope_6_foto_mes_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY comisiones_otras_slope_1_foto_mes) AS comisiones_otras_slope_1_foto_mes_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY comisiones_otras_slope_2_foto_mes) AS comisiones_otras_slope_2_foto_mes_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY comisiones_otras_slope_3_foto_mes) AS comisiones_otras_slope_3_foto_mes_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY comisiones_otras_slope_4_foto_mes) AS comisiones_otras_slope_4_foto_mes_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY comisiones_otras_slope_5_foto_mes) AS comisiones_otras_slope_5_foto_mes_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY comisiones_otras_slope_6_foto_mes) AS comisiones_otras_slope_6_foto_mes_10tile,

    ntile(10) OVER (PARTITION BY foto_mes ORDER BY ratio_saldo_margen) AS ratio_saldo_margen_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY total_comisiones_cliente) AS total_comisiones_cliente_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY transacciones_totales_tarjetas) AS transacciones_totales_tarjetas_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY delta_debito_vs_credito) AS delta_debito_vs_credito_10tile,
    ntile(10) OVER (PARTITION BY foto_mes ORDER BY indicador_banca_digital) AS indicador_banca_digital_10tile

FROM base_features;

,Success


Guardando tabla

In [ ]:
%%sql
COPY competencia_03_fe6x_6 TO "{{dataset_file_fe6_6xxpqt}}" (FORMAT PARQUET);

,Success
